#Sentiment Analysis using BERT pretrained model

In [0]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

#Data

In [0]:
df=pd.read_csv('/content/smile-annotations-final.csv',
               names=['id','text','category'])
df.set_index('id', inplace=True)

#Data Analysis


###Clearing Noise

In [0]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: category, dtype: int64

In [0]:
df=df[~df.category.str.contains('\|')]

In [0]:
df.category.value_counts()

nocode          1572
happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [0]:
df=df[df.category != 'nocode']

###Labelling Data

In [0]:
possibleLabels = df.category.unique()
possibleLabels

array(['happy', 'not-relevant', 'angry', 'disgust', 'sad', 'surprise'],
      dtype=object)

In [0]:
labelDict = {}
for index, possibleLabel in enumerate(possibleLabels):
  labelDict[possibleLabel] = index
labelDict 

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'surprise': 5}

In [0]:
df['label'] = df.category.replace(labelDict)
df

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0
...,...,...,...
611258135270060033,@_TheWhitechapel @Campaignforwool @SlowTextile...,not-relevant,1
612214539468279808,“@britishmuseum: Thanks for ranking us #1 in @...,happy,0
613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy,0


#Training Data

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
xTrain, xVal, yTrain, yVal = train_test_split(
    df.index.values,
    df.label.values,
    test_size = .15,
    random_state = 17,
    stratify = df.label.values
)

In [0]:
df['dataType'] = ['not set']*df.shape[0]

In [0]:
df.loc[xTrain, 'dataType'] = 'train'
df.loc[xVal, 'dataType'] = 'val'

In [0]:
df.groupby(['category', 'label', 'dataType']).count()

text
category     label dataType      
angry        2     train       48
                   val          9
disgust      3     train        5
                   val          1
happy        0     train      966
                   val        171
not-relevant 1     train      182
                   val         32
sad          4     train       27
                   val          5
surprise     5     train       30
                   val          5

# Loading Tokenizer

In [0]:
!pip install transformers

     |████████████████████████████████| 675kB 3.3MB/s 
     |████████████████████████████████| 890kB 17.5MB/s 
     |████████████████████████████████| 1.1MB 14.7MB/s 
     |████████████████████████████████| 3.8MB 31.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=765930fdfbdb872a51b6fe89f19de6ce8969a8f871922ac4f4062e0a1e23388f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [0]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
    )

In [0]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.dataType=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.dataType=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.dataType=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.dataType=='val'].label.values)

In [0]:
dataset_train = TensorDataset( input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset( input_ids_val, attention_masks_val, labels_val)

In [0]:
len(dataset_val)
#len(dataset_train)

223

#Importing Pretrained BERT Model


In [0]:
from transformers import BertForSequenceClassification

In [0]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = len(labelDict),
    output_attentions = False,
    output_hidden_states = False
)

#Data Loaders

In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [0]:
dataLoaderTrain = DataLoader(
    dataset_train,
    sampler = RandomSampler(dataset_train),
    batch_size = 10
)

In [0]:
dataLoaderVal = DataLoader(
    dataset_val,
    sampler = RandomSampler(dataset_val),
    batch_size = 32
)

#Optimizer

In [0]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [0]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-05,
    eps = 1e-8
)

In [0]:
epochs = 1

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = len(dataLoaderTrain)*epoch
)

#Performance Matrices

In [0]:
import numpy as np

In [0]:
from sklearn.metrics import f1_score

In [0]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [0]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in labelDict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

#Training Loop

In [0]:
import random

In [0]:
seedValue = 10
random.seed(seedValue)
np.random.seed(seedValue)
torch.manual_seed(seedValue)
torch.cuda.manual_seed_all(seedValue)

In [89]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
device

device(type='cpu')

In [0]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [74]:
for epoch in tqdm(range(1 , epochs+1)):

  model.train()
  lossTrainTotal = 0

  progressBar = tqdm( 
      dataLoaderTrain,
      desc ='Epoch {:1d}'.format(epoch),
      leave = False,
      disable = False)
  for batch in progressBar:
    model.zero_grad()
    batch = tuple(b.to(device) for b in batch)

    inputs = {
        'input_ids':      batch[0],
        'attention_mask': batch[1],
        'labels':         batch[2],
    }

    outputs = model(**inputs)
    
    loss = outputs[0]
    lossTrainTotal += loss.item()
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    optimizer.step()
    scheduler.step()
        
    progressBar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        
  torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
  tqdm.write(f'\nEpoch {epoch}')
    
  loss_train_avg = lossTrainTotal/len(dataLoaderTrain)            
  tqdm.write(f'Training loss: {loss_train_avg}')
    
  val_loss, predictions, true_vals = evaluate(dataLoaderVal)
  val_f1 = f1_score_func(predictions, true_vals)
  tqdm.write(f'Validation loss: {val_loss}')
  tqdm.write(f'F1 Score (Weighted): {val_f1}')  





Epoch 1
Training loss: 0.40131280201649855
Validation loss: 0.5709036758967808
F1 Score (Weighted): 0.781976051018873



#Loading Evaluating Model

In [0]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(labelDict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)
pass

In [81]:
model.load_state_dict(torch.load('/content/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [0]:
_, predictions, true_vals = evaluate(dataLoaderVal)

In [86]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy: 165/171

Class: not-relevant
Accuracy: 18/32

Class: angry
Accuracy: 0/9

Class: disgust
Accuracy: 0/1

Class: sad
Accuracy: 0/5

Class: surprise
Accuracy: 0/5



In [88]:
#epoch = 1
#batchSize = 32


7